# Graph Analysis Techniques without Feature Learning

# Graph clustering

## Lab 3 : Louvain

### Xavier Bresson  


In [1]:
# For Google Colaboratory
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    path_to_file = '/content/gdrive/My Drive/GML_May23_codes/codes/03_Traditional_GML/01_graph_clustering'
    print(path_to_file)
    # change current path to the folder containing "path_to_file"
    os.chdir(path_to_file)
    !pwd
    !pip install python-louvain==0.15 # install louvain
    

In [2]:
# Load libraries

# Math
import numpy as np

# Import data
import scipy.io

# Visualization 
%matplotlib inline
#%matplotlib notebook 
from matplotlib import pyplot
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
import time

# Import functions in lib folder
import sys
sys.path.insert(0, 'lib/')

# Import helper functions
%load_ext autoreload
%autoreload 2
from lib.utils import construct_kernel
from lib.utils import compute_kernel_kmeans_EM
from lib.utils import compute_kernel_kmeans_spectral
from lib.utils import compute_purity
from lib.utils import construct_knn_graph
from lib.utils import compute_ncut
from lib.utils import compute_pcut
from lib.utils import graph_laplacian

# Louvain algorithm
import community
import networkx as nx

# Remove warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Load MIREX Music dataset
mat = scipy.io.loadmat('datasets/MIREX.mat')
W = mat['W']
n = W.shape[0]
Cgt = mat['Cgt'] - 1; Cgt = Cgt.squeeze()
nc = len(np.unique(Cgt))
print(n,nc)

3090 10


In [4]:
# Baseline accuracy
Crand = np.random.randint(0,nc,[n])
acc = compute_purity(Crand,Cgt,nc)
print(acc)

13.624595469255663


In [5]:
Cncut,acc = compute_ncut(W,Cgt,nc) # 39.06
print(acc)

39.15857605177994


In [6]:
Cpcut,acc = compute_pcut(W,Cgt,nc,1./2,100,False) # 44
print(acc)

38.543689320388346


In [7]:
# Run Louvain

# Louvain partition, Blondel-et.al, Fast unfolding of communities in large networks, 2008
Wnx = nx.from_numpy_array(W.toarray())
partition = community.best_partition(Wnx)
nc_louvain = len(np.unique( [partition[nodes] for nodes in partition.keys()] ))
n = len(Wnx.nodes())
print('nb_data=', n , ', nb_clusters=', nc_louvain)

# Extract clusters
Clouv = np.zeros([n])
clusters = []
k = 0
for com in set(partition.values()):
    list_nodes = [nodes for nodes in partition.keys() if partition[nodes] == com]
    Clouv[list_nodes] = k
    k += 1
    clusters.append(list_nodes)
    
# Accuracy
acc = compute_purity(Clouv,Cgt,nc_louvain)
print('accuracy_louvain=',acc,' with nb_clusters=',nc_louvain)

nb_data= 3090 , nb_clusters= 280
accuracy_louvain= 57.11974110032363  with nb_clusters= 280


In [8]:
# Run NCut with nc_louvain classes
Cncut,acc = compute_ncut(W,Cgt,nc_louvain) #
print(acc)

56.8284789644013


In [9]:
# Load USPS Music dataset
mat = scipy.io.loadmat('datasets/USPS.mat')
W = mat['W']
n = W.shape[0]
Cgt = mat['Cgt'] - 1; Cgt = Cgt.squeeze()
nc = len(np.unique(Cgt))
print(n,nc)

9298 10


In [10]:
# Baseline accuracy
Crand = np.random.randint(0,nc,[n])
acc = compute_purity(Crand,Cgt,nc)
print(acc)

# Run NCut
Cncut,acc = compute_ncut(W,Cgt,nc) 
print(acc)

16.702516670251665
73.52118735211873


In [11]:
# Run Louvain

# Louvain partition, Blondel-et.al, Fast unfolding of communities in large networks, 2008
Wnx = nx.from_numpy_array(W.toarray())
partition = community.best_partition(Wnx)
nc_louvain = len(np.unique( [partition[nodes] for nodes in partition.keys()] ))
n = len(Wnx.nodes())
print('nb_data:', n , ', nb_clusters=', nc_louvain)

# Extract clusters
Clouv = np.zeros([n])
clusters = []
k = 0
for com in set(partition.values()):
    list_nodes = [nodes for nodes in partition.keys() if partition[nodes] == com]
    Clouv[list_nodes] = k
    k += 1
    clusters.append(list_nodes)
    
# Accuracy
acc = compute_purity(Clouv,Cgt,nc_louvain)
print('accuracy_louvain=',acc,' with nb_clusters=',nc_louvain)

nb_data: 9298 , nb_clusters= 16
accuracy_louvain= 95.80554958055497  with nb_clusters= 16


In [12]:
# Run NCut
Cncut,acc = compute_ncut(W,Cgt,nc_louvain)
print(acc)

94.16003441600344
